In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv


load_dotenv()

loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000416134",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body","media_end_head go_trans"]}
        )
    )
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriver = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
"""
당신은 질문-답변을 수행하는 AI 어시스턴트이다.
주어진 문맥에 검색된 다음문맥(context)를 사용해 질문에 답하세요
만약 주어진 문멕에서 답을 찾을 수 없을경우, 모른다고 이야기하세요.
한글로 답변해주세요.

#Question
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model="gpt-4", temperature=0)

news_chain = (
    {"context":retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = news_chain.invoke("로제의 소감은 뭐래?")

print(answer)

"세상에 무슨 일이 일어나고 있나. 이건 미쳤다"며 "넘버원(로제 팬덤), 블링크(블랙핑크 팬덤), 모든 이들에게 감사하다. 내 꿈이 이뤄졌다"라고 소감을 전했습니다.


In [4]:
!pip install bs4

In [6]:
import bs4
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 환경변수 가져오기
load_dotenv()

# 문서 로드
loader = WebBaseLoader(
    web_path=("https://n.news.naver.com/article/437/0000416134"),
    bs_kwargs=dict(
        # 특정 요소에서만 파싱하도록 제한하는 필터 적용
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head go_trans"]}
        )
    )
)

docs = loader.load()

# print(docs)

# 문서분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

splits = text_splitter.split_documents(docs)

# 임베딩 생성
embedding = OpenAIEmbeddings()

# 백터스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

# 검색기 (retriver) 생성
retriver = vectorstore.as_retriever()

# runtime

# 프롬프트 
prompt = PromptTemplate.from_template(
"""
당신은 질문-답변을 수행하는 AI 어시스턴트이다.
주어진 문맥에 검색된 다음문맥(context)를 사용해 질문에 답해야 한다.
만약 주어진 문맥에서 답을 찾을 수 없는경우, 모른다고 이야기하세요.
한글로 답변해주세요.

#Question:
{question}

#Context:
{context}

#Answer:
"""
)

# LLM 모델
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Chain 구성
news_chain = (
    {"context":retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


# 질문
answer = news_chain.invoke("아파트는 누가부른 노래야?")

print(answer)



'아파트'는 블랙핑크의 로제와 팝스타 브루노 마스가 함께 부른 노래입니다.
